<a href="https://colab.research.google.com/github/ShreyaRChittaragi/Crop/blob/main/Crop_weed_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Unzip dataset (replace path as needed)
!unzip "/content/drive/MyDrive/Crop_weed.zip.zip" -d "/content/crop_weed_data"


Mounted at /content/drive
Archive:  /content/drive/MyDrive/Crop_weed.zip.zip
   creating: /content/crop_weed_data/Project5_Ag_Crop and weed detection/
  inflating: /content/crop_weed_data/Project5_Ag_Crop and weed detection/Project5_Ag_Crop and weed detection.docx  
  inflating: /content/crop_weed_data/Project5_Ag_Crop and weed detection/Project5_Ag_Crop and weed detection.zip  
  inflating: /content/crop_weed_data/Project5_Ag_Crop and weed detection/~$oject1.docx  


In [ ]:
import cv2
import matplotlib.pyplot as plt
import os # Import the os module

def draw_boxes(image_path, label_path):
    # Check if the image file exists
    if not os.path.exists(image_path):
        print(f"Error: Image file not found at {image_path}")
        return

    img = cv2.imread(image_path)
    # Add a check to see if the image was loaded successfully
    if img is None:
        print(f"Error: Could not load image from {image_path}")
        return

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w, _ = img.shape

    # Check if the label file exists
    if not os.path.exists(label_path):
        print(f"Error: Label file not found at {label_path}")
        return

    with open(label_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            cls, x, y, bw, bh = map(float, line.strip().split())
            x1 = int((x - bw/2) * w)
            y1 = int((y - bh/2) * h)
            x2 = int((x + bw/2) * w)
            y2 = int((y + bh/2) * h)
            color = (0,255,0) if int(cls)==0 else (255,0,0)
            label = "Crop" if int(cls)==0 else "Weed"
            cv2.rectangle(img, (x1,y1), (x2,y2), color, 2)
            cv2.putText(img, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    plt.figure(figsize=(6,6))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Labeled Image')
    plt.show()

# Change name accordingly
draw_boxes('/content/crop_weed_data/images/image1.jpg', '/content/crop_weed_data/labels/image1.txt')

Error: Image file not found at /content/crop_weed_data/images/image1.jpg


In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

import os
from sklearn.model_selection import train_test_split
from glob import glob

# Correcting the path based on the unzip output
images = glob('/content/crop_weed_data/Project5_Ag_Crop and weed detection/agri_data/data/*.jpeg')
train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

def write_paths(paths, filename):
    with open(filename, 'w') as f:
        f.write('\n'.join(paths))

write_paths(train_imgs, '/content/crop_weed_data/train.txt')
write_paths(val_imgs, '/content/crop_weed_data/val.txt')

Cloning into 'yolov5'...
remote: Enumerating objects: 17488, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 17488 (delta 0), reused 0 (delta 0), pack-reused 17486 (from 2)
Receiving objects: 100% (17488/17488), 16.54 MiB | 17.02 MiB/s, done.
Resolving deltas: 100% (11988/11988), done.
/content/yolov5/yolov5/yolov5


In [ ]:
yaml_content = """
path: /content/crop_weed_data
train: train.txt
val: val.txt

names:
  0: Crop
  1: Weed
nc: 2
"""

with open('/content/crop_weed_data/cropweed.yaml', 'w') as f:
    f.write(yaml_content)


In [ ]:
!python train.py --img 512 --batch 16 --epochs 100 --data /content/crop_weed_data/cropweed.yaml --weights yolov5s.pt --device 0


Streaming output truncated to the last 5000 lines.
      63/99      2.85G    0.02968    0.01762   0.002468         42        512:  58% 38/65 [00:12<00:08,  3.13it/s]/content/yolov5/yolov5/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      63/99      2.85G     0.0297    0.01756   0.002423         41        512:  60% 39/65 [00:13<00:07,  3.32it/s]/content/yolov5/yolov5/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      63/99      2.85G     0.0298    0.01758   0.002423         56        512:  62% 40/65 [00:13<00:07,  3.28it/s]/content/yolov5/yolov5/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(a

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --source /content/crop_weed_data/Project5_Ag_Crop\ and\ weed\ detection/agri_data/data/




detect: weights=['runs/train/exp/weights/best.pt'], source=/content/crop_weed_data/Project5_Ag_Crop and weed detection/agri_data/data/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-420-g0c99ce80 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1300 /content/crop_weed_data/Project5_Ag_Crop and weed detection/agri_data/data/agri_0_1009.jpeg: 640x640 1 Weed, 11.4ms
image 2/1300 /content/crop_weed_data/Project5_Ag_Crop and weed detection/agri_data/data/agri_0_1017.jpeg: 640x640 3 Weeds, 11.5ms

In [ ]:
!ls -R "/content/crop_weed_data/Project5_Ag_Crop and weed detection/"

'/content/crop_weed_data/Project5_Ag_Crop and weed detection/':
'~$oject1.docx'
'Project5_Ag_Crop and weed detection.docx'
'Project5_Ag_Crop and weed detection.zip'


In [ ]:
!unzip "/content/crop_weed_data/Project5_Ag_Crop and weed detection/Project5_Ag_Crop and weed detection.zip" -d "/content/crop_weed_data/Project5_Ag_Crop and weed detection/"

Archive:  /content/crop_weed_data/Project5_Ag_Crop and weed detection/Project5_Ag_Crop and weed detection.zip
  inflating: /content/crop_weed_data/Project5_Ag_Crop and weed detection/agri_data/data/agri_0_1009.jpeg  
  inflating: /content/crop_weed_data/Project5_Ag_Crop and weed detection/agri_data/data/agri_0_1009.txt  
  inflating: /content/crop_weed_data/Project5_Ag_Crop and weed detection/agri_data/data/agri_0_1017.jpeg  
  inflating: /content/crop_weed_data/Project5_Ag_Crop and weed detection/agri_data/data/agri_0_1017.txt  
  inflating: /content/crop_weed_data/Project5_Ag_Crop and weed detection/agri_data/data/agri_0_1018.jpeg  
  inflating: /content/crop_weed_data/Project5_Ag_Crop and weed detection/agri_data/data/agri_0_1018.txt  
  inflating: /content/crop_weed_data/Project5_Ag_Crop and weed detection/agri_data/data/agri_0_1020.jpeg  
  inflating: /content/crop_weed_data/Project5_Ag_Crop and weed detection/agri_data/data/agri_0_1020.txt  
  inflating: /content/crop_weed_data/P

In [ ]:
!mkdir -p /content/images
!cp /content/crop_weed_data/Project5_Ag_Crop\ and\ weed\ detection/agri_data/data/*.jpeg /content/images/


In [ ]:
!ls /content/images | head


agri_0_1009.jpeg
agri_0_1017.jpeg
agri_0_1018.jpeg
agri_0_1020.jpeg
agri_0_1024.jpeg
agri_0_1026.jpeg
agri_0_1028.jpeg
agri_0_1041.jpeg
agri_0_1053.jpeg
agri_0_1068.jpeg


In [ ]:
!find /content -name best.pt


/content/yolov5/yolov5/yolov5/runs/train/exp/weights/best.pt


In [ ]:
!python /content/yolov5/yolov5/yolov5/detect.py \
--weights /content/yolov5/yolov5/yolov5/runs/train/exp/weights/best.pt \
--source /content/images \
--imgsz 512 \
--conf-thres 0.25 \
--device 0 \
--save-conf \
--save-txt


detect: weights=['/content/yolov5/yolov5/yolov5/runs/train/exp/weights/best.pt'], source=/content/images, data=data/coco128.yaml, imgsz=[512, 512], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-420-g0c99ce80 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1300 /content/images/agri_0_1009.jpeg: 512x512 1 Weed, 7.6ms
image 2/1300 /content/images/agri_0_1017.jpeg: 512x512 3 Weeds, 7.7ms
image 3/1300 /content/images/agri_0_1018.jpeg: 512x512 2 Weeds, 7.6ms
image 4/1300 /content/images/agri_0_1020.jpeg: 512x512 1 Weed, 7.6ms
image 5/130

In [ ]:
import os

detect_dir = "/content/yolov5/yolov5/yolov5/runs/detect"
print("Subdirectories in runs/detect:")
print(os.listdir(detect_dir))


Subdirectories in runs/detect:
['exp3', 'exp2', 'exp', 'exp4', 'exp5']


In [ ]:
from IPython.display import Image, display
import glob

img_paths = glob.glob('/content/yolov5/yolov5/yolov5/runs/detect/exp4/*.jpeg')
for img_path in img_paths:
    display(Image(filename=img_path))
